In [2]:
from google.colab import files
uploaded = files.upload()

Saving chest_xray.rar to chest_xray.rar


In [5]:
import os

rar_path = "chest_xray.rar"  # Yüklediğiniz dosya adı
extract_path = "/content/chest_xray"

os.makedirs(extract_path, exist_ok=True)

!unrar x "{rar_path}" "{extract_path}"

print(f"✔ .rar dosyası çıkarıldı: {extract_path}")


Görüntülenen çıkış son 5000 satıra kısaltıldı.
Extracting  /content/chest_xray/chest_xray/train/bacteria/person1771_bacteria_4615.jpeg      10%  OK 
Extracting  /content/chest_xray/chest_xray/train/bacteria/person1779_bacteria_4626.jpeg      10%  OK 
Extracting  /content/chest_xray/chest_xray/train/bacteria/person1787_bacteria_4634.jpeg      10%  OK 
Extracting  /content/chest_xray/chest_xray/train/bacteria/person1796_bacteria_4644.jpeg      10%  OK 
Extracting  /content/chest_xray/chest_xray/train/bacteria/person1799_bacteria_4647.jpeg      10%  OK 
Extracting  /content/chest_xray/chest_xray/train/bacteria/person17_bacteria_56.jpeg      10%  OK 
Extracting  /content/chest_xray/chest_xray/train/bacteria/person1810_bacteria_4664.jpeg      10%  OK 
Extracting  /content/chest_xray/chest_xray/train/bacteria/person1812_bacteria_4667.jpeg      10%  OK 
Extracting  /content/chest_xray/chest_xray/train/bacteria/person1823_

In [8]:
import os

# Sınıf isimlerini etiketle
class_mapping = {
    "bacteria": 0,
    "virus": 1,
    "NORMAL": 2  # Eğer bbox yoksa buna gerek olmayabilir
}

image_dir = "chest_xray/chest_xray/train"
label_dir = os.path.join(image_dir, "labels")

for class_name in ["bacteria", "virus"]:
    class_id = class_mapping[class_name]
    class_folder = os.path.join(image_dir, class_name)

    for img_file in os.listdir(class_folder):
        base_name = os.path.splitext(img_file)[0]
        label_path = os.path.join(label_dir, f"{base_name}.txt")

        if os.path.exists(label_path):
            with open(label_path, "r") as f:
                lines = f.readlines()

            # Sınıf ID'sini güncelle
            updated_lines = []
            for line in lines:
                parts = line.strip().split()
                if len(parts) == 5:
                    parts[0] = str(class_id)
                    updated_lines.append(" ".join(parts))

            # Dosyayı tekrar yaz
            with open(label_path, "w") as f:
                f.write("\n".join(updated_lines))


In [31]:
import os

# Yol bilgileri - senin klasör yapına tam uyumlu
normal_dir = '/content/chest_xray/chest_xray/train/NORMAL'
labels_dir = '/content/chest_xray/chest_xray/train/labels'

# YOLO formatı için sabit bbox bilgisi (normalize edilmiş)
class_id = 2
x_center = 0.5
y_center = 0.5
width = 0.5
height = 0.5

# labels klasörünü oluştur (varsa hata vermez)
os.makedirs(labels_dir, exist_ok=True)

# NORMAL klasöründeki görüntülerin her biri için etiket dosyası oluştur
for filename in os.listdir(normal_dir):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        base_name = os.path.splitext(filename)[0]
        label_path = os.path.join(labels_dir, base_name + '.txt')

        with open(label_path, 'w') as f:
            # YOLO formatında bbox satırı yazılır
            f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

print("NORMAL klasörü için etiket dosyaları başarıyla oluşturuldu.")


NORMAL klasörü için etiket dosyaları başarıyla oluşturuldu.


In [32]:
!git clone https://github.com/ultralytics/yolov5  # YOLOv5'i indir
%cd yolov5
%pip install -r requirements.txt  # Gereksinimleri yükle

Cloning into 'yolov5'...
remote: Enumerating objects: 17485, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 17485 (delta 81), reused 31 (delta 31), pack-reused 17374 (from 3)
Receiving objects: 100% (17485/17485), 16.39 MiB | 19.42 MiB/s, done.
Resolving deltas: 100% (11986/11986), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 128.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/

In [33]:
import os
import shutil

# paths
source_images = '/content/chest_xray/chest_xray/test'
destination = '/content/yolov5/data/chest_xray'

# klasörleri sıfırla
if os.path.exists(destination):
    shutil.rmtree(destination)

os.makedirs(f"{destination}/images", exist_ok=True)
os.makedirs(f"{destination}/labels", exist_ok=True)

# sadece .jpg/.png uzantılı dosyaları kopyala
for subfolder in ['NORMAL', 'bacteria', 'virus']:
    subdir = os.path.join(source_images, subfolder)
    for file in os.listdir(subdir):
        if file.endswith(('.jpg', '.jpeg', '.png')):
            base = os.path.splitext(file)[0]
            shutil.copy(os.path.join(subdir, file), os.path.join(destination, 'images', file))

# etiketleri kopyala
labels_src = os.path.join(source_images, 'labels')
for file in os.listdir(labels_src):
    if file.endswith('.txt'):
        shutil.copy(os.path.join(labels_src, file), os.path.join(destination, 'labels', file))

print("Görüntüler ve etiketler kopyalandı.")


Görüntüler ve etiketler kopyalandı.


In [48]:
data_yaml = """
train: data/chest_xray/images
val: data/chest_xray/images

nc: 3
names: ['bacteria', 'virus', 'normal']
"""

with open('/content/yolov5/data/chest_xray.yaml', 'w') as f:
    f.write(data_yaml)


In [46]:
import os
print("Dosya var mı?", os.path.exists("/content/chest_xray.yaml"))


Dosya var mı? True


In [36]:
!pip install ultralytics
import os
import pandas as pd
from ultralytics import YOLO


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [49]:
!python train.py --img 416 --batch 16 --epochs 100 --data data/chest_xray.yaml --cfg models/yolov5s.yaml --weights yolov5s.pt --name chest_xray_yolo


Görüntülenen çıkış son 5000 satıra kısaltıldı.
  with torch.cuda.amp.autocast(amp):
      12/99      2.39G    0.03843    0.01404   0.005097         35        416:  54% 14/26 [00:05<00:06,  1.84it/s]/content/yolov5/train.py:413: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      12/99      2.39G    0.03812    0.01398   0.004915         37        416:  58% 15/26 [00:06<00:04,  2.29it/s]/content/yolov5/train.py:413: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      12/99      2.39G    0.03778    0.01389   0.004987         33        416:  62% 16/26 [00:07<00:06,  1.66it/s]/content/yolov5/train.py:413: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
    

In [56]:
from google.colab import files
files.download('/content/yolov5/runs/train/chest_xray_yolo/weights/best.pt')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>